<a href="https://colab.research.google.com/github/ValeriaYashan/Machine-learning/blob/main/Detecci%C3%B3n_gatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def descargar_imagenes_de_gatos(numero_imagenes):
    # URL de la API de gatos
    url_api = "https://api.thecatapi.com/v1/images/search"
    # Carpeta donde se guardarán las imágenes descargadas
    carpeta_destino = "imagenes_gatos"
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    # Realizar solicitudes a la API para descargar las imágenes de gatos
    for i in range(numero_imagenes):
        try:
            # Realizar la solicitud GET a la API
            respuesta = requests.get(url_api)
            # Verificar si la solicitud fue exitosa
            if respuesta.status_code == 200:
                # Obtener la URL de la imagen del JSON de respuesta
                url_imagen = respuesta.json()[0]['url']
                # Obtener el nombre de archivo de la URL de la imagen
                nombre_archivo = os.path.join(carpeta_destino, f"gato_{i+1}.jpg")
                # Descargar la imagen y guardarla en la carpeta destino
                with open(nombre_archivo, 'wb') as archivo:
                    archivo.write(requests.get(url_imagen).content)
                print(f"Imagen {i+1} descargada correctamente como '{nombre_archivo}'")
            else:
                print(f"Error al descargar la imagen {i+1}: {respuesta.status_code}")
        except Exception as e:
            print(f"Error al descargar la imagen {i+1}: {e}")

# Especificar el número de imágenes de gatos que deseas descargar
numero_imagenes = 5

# Llamar a la función para descargar las imágenes de gatos
descargar_imagenes_de_gatos(numero_imagenes)

Imagen 1 descargada correctamente como 'imagenes_gatos/gato_1.jpg'
Imagen 2 descargada correctamente como 'imagenes_gatos/gato_2.jpg'
Imagen 3 descargada correctamente como 'imagenes_gatos/gato_3.jpg'
Imagen 4 descargada correctamente como 'imagenes_gatos/gato_4.jpg'
Imagen 5 descargada correctamente como 'imagenes_gatos/gato_5.jpg'


In [19]:
# Ruta del directorio de entrenamiento
directorio_entrenamiento = "/content/imagenes_gatos"

# Dimensiones de las imágenes de entrada
altura, anchura = 150, 150
input_shape = (altura, anchura, 3)

# Crear generador de datos de entrenamiento
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directorio_entrenamiento,
    target_size=(altura, anchura),
    batch_size=20,
    class_mode='binary',
    shuffle=True  # Mezclar las imágenes aleatoriamente
)

# Construir el modelo CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Found 0 images belonging to 0 classes.


In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Construir el modelo CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Visualizar la arquitectura del modelo
model.summary()

# Entrenar el modelo con los datos
# (Aquí deberías proporcionar tus datos de entrenamiento)
# Ejemplo:
# model.fit(train_data, epochs=10, validation_data=validation_data)

# Después de entrenar el modelo, puedes guardarlo para su uso posterior
model.save("modelo_gatos.h5")

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_28 (MaxPooli  (None, 74, 74, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_29 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_29 (MaxPooli  (None, 36, 36, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_30 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_30 (MaxPooli  (None, 17, 17, 128)      

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [21]:
print(os.listdir(directorio_entrenamiento))

['gato_1.jpg', 'gato_2.jpg', 'gato_3.jpg', 'gato_4.jpg', 'gato_5.jpg']


In [24]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from google.colab import files
import io  # Importar el módulo io

# Cargar el modelo entrenado
modelo_entrenado = load_model("modelo_gatos.h5")

def predecir_imagen(imagen):
    # Redimensionar la imagen a 150x150 píxeles (como se utilizó durante el entrenamiento)
    imagen = imagen.resize((150, 150))
    # Convertir la imagen a un array numpy
    imagen_array = np.array(imagen) / 255.0
    imagen_array = np.expand_dims(imagen_array, axis=0)  # Agregar una dimensión adicional para el lote
    # Realizar la predicción
    predicciones = modelo_entrenado.predict(imagen_array)
    return predicciones[0][0]  # Retornar la probabilidad de que la imagen sea un gato

# Solicitar al usuario que suba una imagen
print("Por favor, selecciona la imagen que deseas verificar:")
archivos = files.upload()

# Procesar la imagen subida
for nombre_archivo, contenido_archivo in archivos.items():
    imagen = Image.open(io.BytesIO(contenido_archivo))
    # Realizar la predicción
    probabilidad_gato = predecir_imagen(imagen)
    if probabilidad_gato > 0.5:
        print(f"La imagen '{nombre_archivo}' parece ser un gato con una probabilidad del {probabilidad_gato*100:.2f}%.")
    else:
        print(f"La imagen '{nombre_archivo}' parece no ser un gato con una probabilidad del {(1-probabilidad_gato)*100:.2f}%.")

Por favor, selecciona la imagen que deseas verificar:


Saving Tigre2.jfif to Tigre2.jfif
1/1 [==============================] - 0s 104ms/step
La imagen 'Tigre2.jfif' parece no ser un gato con una probabilidad del 51.60%.
